## This notebook runs optimization experiments on different combination of input/target normalization image transforms using the FNet model architecture by fixing hyper parameters during training to understand the effect of normalization methods on model performance.

In [1]:
import pathlib
import sys
import subprocess
import yaml
import gc

import pandas as pd
import torch
import torch.optim as optim
import mlflow
import optuna
import joblib

## Import global path/model setting variables from config
First retrieves repo root directory, then imports the config.py from repo root
All filepath checks are centrally handled within the config module

In [2]:
def get_repo_root():
    return subprocess.run(
        ["git", "rev-parse", "--show-toplevel"],
        stdout=subprocess.PIPE,
        check=True,
        text=True
    ).stdout.strip()

repo_root = get_repo_root()
sys.path.append(repo_root)

from config import (
    SOFTWARE_PATH,
    LOADDATA_FILE_PATH,
    SC_FEATURES_DIR,
    INPUT_CHANNEL_NAMES,
    TARGET_CHANNEL_NAMES,
)

## Import virtual_stain_flow software 

In [3]:
sys.path.append(str(SOFTWARE_PATH))

## Dataset
from virtual_stain_flow.datasets.PatchDataset import PatchDataset
from virtual_stain_flow.datasets.CachedDataset import CachedDataset

## FNet training
from virtual_stain_flow.models.fnet import FNet
from virtual_stain_flow.trainers.Trainer import Trainer

from virtual_stain_flow.transforms.MinMaxNormalize import MinMaxNormalize
from virtual_stain_flow.transforms.PixelDepthTransform import PixelDepthTransform
from virtual_stain_flow.transforms.ZScoreNormalize import ZScoreNormalize

## Metrics
from virtual_stain_flow.metrics.MetricsWrapper import MetricsWrapper
from virtual_stain_flow.metrics.PSNR import PSNR
from virtual_stain_flow.metrics.SSIM import SSIM

## callback
from virtual_stain_flow.callbacks.MlflowLogger import MlflowLogger

/home/weishanli/anaconda3/envs/speckle_analysis/lib/python3.11/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## Define output paths and model training parameters

In [4]:
## Output directories
MLFLOW_DIR = pathlib.Path('.').absolute() / 'optuna_mlflow'
MLFLOW_DIR.mkdir(parents=True, exist_ok=True)

# dump directory for optuna studies
OPTUNA_JOBLIB_DIR = pathlib.Path('.').absolute() / 'optuna_joblib'
OPTUNA_JOBLIB_DIR.mkdir(parents=True, exist_ok=True)

## Basic data generation, model convolutional depth, optimizer param and max epoch definition
# Will be using these fixed values for the normalization method optimization
PATCH_SIZE = 256
CONV_DEPTH = 5
LR = 1e-4
BETAS = (0.5, 0.9)
BATCH_SIZE = 16
EPOCHS = 1_000
PATIENCE = 20

## Configure Normalization Transforms

In [5]:
## Define transforms and parameters
NORM_METHODS = {
    "z_score": {
        "class": ZScoreNormalize,
        "args": {"_mean": None, "_std": None, "_always_apply": True, "_p": 1.0}
    },
    "8bit": {
        "class": PixelDepthTransform,
        "args": {"src_bit_depth": 16, "target_bit_depth": 8, "_always_apply": True, "_p": 1.0}
    },
    "min_max": {
        "class": MinMaxNormalize,
        "args": {"_normalization_factor": (2 ** 16) - 1, "_always_apply": True, "_p": 1.0}
    }
}

## Define the model output activation to be used with each output normalization
NORM_METHOD_ACTIVATION = {
    "z_score": "linear",
    "8bit": "linear",
    "min_max": "sigmoid"
}

## Define optimization objective functions

In [6]:
CACHE_DATA = True

def free_gpu_memory():
    gc.collect()
    torch.cuda.empty_cache()

def objective(trial, dataset, channel_name):

    # Suggest an input and targettransform
    input_transform = trial.suggest_categorical("input_transform", list(NORM_METHODS.keys()))
    target_transform = trial.suggest_categorical("target_transform", list(NORM_METHODS.keys()))

    ## Configure the dataset with normalization methods
    dataset.set_input_transform(NORM_METHODS[input_transform]["class"](**NORM_METHODS[input_transform]["args"]))
    dataset.set_target_transform(NORM_METHODS[target_transform]["class"](**NORM_METHODS[target_transform]["args"]))

    ## Cache dataset
    # Caching PatchDatasets (into RAM) can substantially improve training speed, mostly
    # due to speeding up the data shuffling process that can be slow with dynamically
    # cropping patches from large images. However, to really benefit from caching it is
    # necessary to use a cache size that fits the entire dataset (or close to doing so).
    # Consider not using the Cached Dataset if memory is limited. Training/optimization is 
    # completely functional with the dynamic PatchDataset.
    if CACHE_DATA:
        dataset = CachedDataset(
                dataset=dataset,
                prefill_cache=True
            )
    else:
        # uses the dynamic PatchDataset
        pass

    ## Setup model and optimizer
    model = FNet(depth=CONV_DEPTH, 
                 # output activation paired with target/output normalization
                 output_activation=NORM_METHOD_ACTIVATION[target_transform])
    optimizer = optim.Adam(model.parameters(), lr=LR, betas=BETAS)
    
    ## Metrics to be computed (and logged)
    metric_fns = {
        "mse_loss": MetricsWrapper(_metric_name='mse', module=torch.nn.MSELoss()),
        "ssim_loss": SSIM(_metric_name="ssim"),
        "psnr_loss": PSNR(_metric_name="psnr"),
    }

    ## Params to log with mlflow
    params = {
            "lr": LR,
            "beta0": BETAS[0],
            "beta1": BETAS[1],
            "depth": CONV_DEPTH,
            "patch_size": PATCH_SIZE,
            "batch_size": BATCH_SIZE,
            "epochs": EPOCHS,
            "patience": PATIENCE,
            "input_norm": input_transform,
            "target_norm": target_transform,
            "channel_name": channel_name,
        }

    ## mlflow logger callback
    mlflow_logger_callback = MlflowLogger(
        name='mlflow_logger',
        mlflow_uri=MLFLOW_DIR / 'mlruns',
        mlflow_experiment_name='FNet_optimize_io_norm',
        mlflow_start_run_args={'run_name': f'FNet_optimize_io_norm_{channel_name}', 'nested': True},
        mlflow_log_params_args=params
    )
    
    ## Trainer
    trainer = Trainer(
        model = model,
        optimizer = optimizer,
        backprop_loss = torch.nn.L1Loss(), # MAE loss for backpropagation
        dataset = dataset,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        patience = PATIENCE,
        callbacks=[mlflow_logger_callback],
        metrics=metric_fns,
        device = 'cuda',
        early_termination_metric='L1Loss'
    )

    # Train the model and log validation loss
    trainer.train()
    val_loss = trainer.best_loss

    del model
    del optimizer
    del metric_fns
    del mlflow_logger_callback
    del trainer
    
    free_gpu_memory()

    return val_loss

## Optimize for I/O normalizationm method per Channel

In [7]:
N_TRIALS = 100

## Loaddata for optimization
loaddata_df = pd.read_csv(LOADDATA_FILE_PATH)
sc_features = pd.DataFrame()

## Retrieve relevant sc features by assemblying them from parquet files
for plate in loaddata_df['Metadata_Plate'].unique():
    sc_features_parquet = SC_FEATURES_DIR / f'{plate}_sc_normalized.parquet'
    if not sc_features_parquet.exists():
        print(f'{sc_features_parquet} does not exist, skipping...')
        continue 
    else:
        sc_features = pd.concat([
            sc_features, 
            pd.read_parquet(
                sc_features_parquet,
                columns=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site', 'Metadata_Cells_Location_Center_X', 'Metadata_Cells_Location_Center_Y']
            )
        ])

## Create patch dataset
pds = PatchDataset(
        _loaddata_csv=loaddata_df,
        _sc_feature=sc_features,
        _input_channel_keys=INPUT_CHANNEL_NAMES,
        _target_channel_keys=TARGET_CHANNEL_NAMES,
        _input_transform=None,
        _target_transform=None,
        patch_size=PATCH_SIZE,
        verbose=False,
        patch_generation_method="random_cell",
        n_expected_patches_per_img=50,
        patch_generation_random_seed=42
    )

for channel_name in TARGET_CHANNEL_NAMES:

    ## Configure dataset channel
    pds.set_input_channel_keys(INPUT_CHANNEL_NAMES)
    pds.set_target_channel_keys(channel_name)
    ## Caching of dataset is handled within the objective function due 
    ## to the need to change normalization methods for each trial

    print(f"Beginning optimization for channel: {channel_name} for io normalization methods")

    # Load the existing study
    study_path = OPTUNA_JOBLIB_DIR / f"FNet_optimize_{channel_name}_io_norm.joblib"
    if study_path.exists():
        study = joblib.load(study_path)
    else:
        # Or create if not already existing
        study = optuna.create_study(
            direction="minimize",
            study_name=f"FNet_optimize_{channel_name}_io_norm",
            sampler=optuna.samplers.TPESampler(seed=42)
        )

    # Resume optimization and run up until N_TRIALS
    while len(study.trials) < N_TRIALS:
        study.optimize(lambda trial: objective(trial, pds, channel_name), n_trials=1)
        joblib.dump(study, study_path)
        print(f"Saved study after trial {len(study.trials)}/{N_TRIALS}")
    
    print(f"{N_TRIALS} of Normalization Method Optimization for {channel_name} completed.")

    # Print best trial results
    print(f"Best trial for channel {channel_name}:")
    print(f"  Validation Loss: {study.best_trial.value}")
    print(f"  Hyperparameters: {study.best_trial.params}")

Beginning optimization for channel: OrigDNA for io normalization methods
Early termination at epoch 37 with best validation metric 0.011204013717360795


[I 2025-06-17 11:20:02,483] Trial 50 finished with value: 0.011204013717360795 and parameters: {'input_transform': '8bit', 'target_transform': 'min_max'}. Best is trial 20 with value: 0.0048707767855376005.


Saved study after trial 51/100
Early termination at epoch 33 with best validation metric 0.011426672237575985


[I 2025-06-17 11:30:02,436] Trial 51 finished with value: 0.011426672237575985 and parameters: {'input_transform': '8bit', 'target_transform': 'min_max'}. Best is trial 20 with value: 0.0048707767855376005.


Saved study after trial 52/100
Early termination at epoch 75 with best validation metric 0.011198313441127539


...

Saved study after trial 63/100
Early termination at epoch 79 with best validation metric 0.017185176780913025


[I 2025-06-18 09:49:30,398] Trial 63 finished with value: 0.017185176780913025 and parameters: {'input_transform': 'z_score', 'target_transform': 'min_max'}. Best is trial 16 with value: 0.004642733547370881.


Saved study after trial 64/100


[W 2025-06-18 09:56:40,246] Trial 64 failed with parameters: {'input_transform': 'z_score', 'target_transform': 'min_max'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/weishanli/anaconda3/envs/speckle_analysis/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_23237/989379415.py", line 61, in <lambda>
    study.optimize(lambda trial: objective(trial, pds, channel_name), n_trials=1)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_23237/1915596034.py", line 86, in objective
    trainer.train()
  File "/home/weishanli/Waylab/virtual_stain_flow/trainers/AbstractTrainer.py", line 193, in train
    train_loss = self.train_epoch()
                 ^^^^^^^^^^^^^^^^^^
  File "/home/weishanli/Waylab/virtual_stain_flow/trainers/Trainer.py", line 119, in train_epoch
    batch_los

KeyboardInterrupt: 